In [1]:
# ✅ Milestone 1: Install dependencies and set up environment
!pip install -q sentence-transformers faiss-cpu transformers tqdm



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.4 MB/s eta 0:00:00


In [2]:
# ✅ Milestone 2: Import libraries
import os
import json
import torch
import faiss
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from transformers import CLIPProcessor, CLIPModel
from PIL import Image


In [3]:
# ✅ Milestone 3: Load and prepare your dataset
# (Make sure you've uploaded your files to the Colab file system first)
from google.colab import files
uploaded = files.upload()  # Upload your .tar file or loose image/subtitle files
import tarfile
with tarfile.open("youtube_dataset.tar", "r") as tar:
    tar.extractall("data")

Saving youtube_dataset.tar to youtube_dataset.tar


In [4]:

# ✅ Milestone 4: Chunk subtitles for embedding
subtitle_chunks = []
subtitle_paths = []

for root, dirs, files in os.walk("data"):
    for file in files:
        if file.endswith(".txt"):
            path = os.path.join(root, file)
            with open(path, 'r') as f:
                lines = f.readlines()
                for i in range(0, len(lines), 5):  # chunks of 5 lines
                    chunk = " ".join([l.strip() for l in lines[i:i+5]])
                    subtitle_chunks.append(chunk)
                    subtitle_paths.append(path)




In [5]:
# ✅ Milestone 5: Embed subtitles
text_model = SentenceTransformer("all-MiniLM-L6-v2")
text_embeddings = text_model.encode(subtitle_chunks, show_progress_bar=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches: 0it [00:00, ?it/s]

In [9]:
print("Chunks:", subtitle_chunks)
print("Number of chunks:", len(subtitle_chunks))


Chunks: []
Number of chunks: 0


In [11]:
# ✅ Milestone 6: Build a FAISS index
print("Shape of text_embeddings:", text_embeddings.shape)
index = faiss.IndexFlatL2(text_embeddings.shape[1])
index.add(text_embeddings)



Shape of text_embeddings: (0,)


IndexError: tuple index out of range

In [7]:
# ✅ Milestone 7: Ask questions (Text-only)
def ask_question(question):
    query_embedding = text_model.encode([question])
    D, I = index.search(query_embedding, k=3)
    for rank, idx in enumerate(I[0]):
        print(f"Top {rank+1} match:\n{subtitle_chunks[idx]}\n")

# Ask your 3 questions
print("\n--- Question 1: What is a convolutional neural network (CNN)?")
ask_question("What is a convolutional neural network (CNN)?")

print("\n--- Question 2: What is the vanishing gradient problem?")
ask_question("What is the vanishing gradient problem?")

print("\n--- Question 3: How do residual connections help in deep learning?")
ask_question("How do residual connections help in deep learning?")



--- Question 1: What is a convolutional neural network (CNN)?


NameError: name 'index' is not defined

In [10]:
# ✅ Milestone 8 (Optional): Add CLIP image embeddings (multimodal)
image_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

image_paths = []
image_embeddings = []

for root, dirs, files in os.walk("data"):
    for file in files:
        if file.endswith(('.jpg', '.png')):
            path = os.path.join(root, file)
            image = Image.open(path).convert("RGB")
            inputs = processor(images=image, return_tensors="pt")
            with torch.no_grad():
                embedding = image_model.get_image_features(**inputs)
            image_embeddings.append(embedding[0].numpy())
            image_paths.append(path)

if image_embeddings:
    image_index = faiss.IndexFlatL2(image_embeddings[0].shape[0])
    image_index.add(np.stack(image_embeddings))

    def search_images(text_query):
        inputs = processor(text=[text_query], return_tensors="pt", padding=True)
        with torch.no_grad():
            text_feat = image_model.get_text_features(**inputs).numpy()
        D, I = image_index.search(text_feat, k=3)
        for i in I[0]:
            print(f"Matching image: {image_paths[i]}")

    print("\n--- Image search: Show me diagrams of CNNs")
    search_images("Show me diagrams of CNNs")



config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]